Analytics Programming — D598  
QKN1 — QKN1 Task 2: Coding  
Student - John D. Pickering    
Programming Language - Python  
Other Information: Panda and NumPy  
Date: 9/12/2025

#### Table of Contents  
Task 2
- Step 1:  Import the data file into a data frame. 
- Step 2:  Identify any duplicate rows in the data set.
- Step 3:  Group all IDs by state, then run descriptive statistics (mean, median, min, & max) for all numeric variables by state and store this result as a new data frame
-  Step 4:  Filter the data frame to identify all businesses with debt-to-equity ratios that are negative.
-  Step 5:  Create a new data frame that provides the debt-to-income ratio for every business in the data set. Debt-to-income ratio is defined as long-term debt divided by revenue.
-  Step 6:  Concatenate the debt-to-income ratio data frame you created with the original data frame.  

### A: Create a program in Gitlab using Python or R to perform the data analysis described in Task 1.

#### Import Dependencies

In [1]:
# -----------------------------
# Import Dependencies
# -----------------------------
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore
import warnings

warnings.filterwarnings('ignore')

#### Import 'D598 Data Set.xlsx' Into Pandas Dataframe

In [2]:
# -----------------------------
# Step 1. Import the data file into a data frame.
# -----------------------------
df = pd.read_excel('D598 Data Set.xlsx')
print('Dataset has been imported')


Dataset has been imported


#### Identify Duplicate Rows

In [3]:
# -----------------------------
# Step 2. Identify any duplicate rows in the data set.
# Send duplicates to Excel for review. 
# -----------------------------
duplicates = df[df.duplicated()]

print("Duplicate Rows:")
if duplicates.empty:
    print("0 duplicate rows found.")
else:
    print(f"{len(duplicates)} duplicate rows found. Exporting to Excel for review...")
    # Export duplicate rows to an Excel file
    duplicates.to_excel("duplicates_review.xlsx", index=False)
    print("Duplicate rows exported to 'duplicates_review.xlsx'")

Duplicate Rows:
0 duplicate rows found.


#### Group all Business IDs by state.  Run descriptive Statistics

In [4]:
# -----------------------------
# Step 3. Group all IDs by state, then run descriptive statistics (mean, median, min, & max) 
# for all numeric variables by state and store this result as a new data frame
# -----------------------------
grouped_stats = df.groupby("Business State").agg({
    "Total Long-term Debt": ["mean", "median", "min", "max"],
    "Total Equity": ["mean", "median", "min", "max"],
    "Debt to Equity": ["mean", "median", "min", "max"],
    "Total Liabilities": ["mean", "median", "min", "max"],
    "Total Revenue": ["mean", "median", "min", "max"],
    "Profit Margin": ["mean", "median", "min", "max"],
})

# Reorder MultiIndex so variables come first, then stats
grouped_stats = grouped_stats.swaplevel(axis=1).sort_index(axis=1)

# Style for readability
styled = (
    grouped_stats.style
    .format("{:,.2f}")                           # commas + 2 decimals
    .set_caption("Descriptive Statistics by State")  # table caption
    .highlight_min(color="lightcoral", axis=0)   # highlight min values
    .highlight_max(color="lightgreen", axis=0)   # highlight max values
    .background_gradient(cmap="Blues", axis=0)   # gradient shading
)

styled

#### Filter Data frame to identify negative debt-to-equity

In [5]:
# -----------------------------
# Step 4. Filter the data frame to identify all businesses with debt-to-equity ratios that are negative.
# -----------------------------
negative_de_ratio = df[df["Debt to Equity"] < 0]

print("\nBusinesses with Negative Debt-to-Equity Ratios:")
print(negative_de_ratio)


Businesses with Negative Debt-to-Equity Ratios:
     Business ID Business State  Total Long-term Debt  Total Equity  \
18     934562013           Ohio           263880000.0  -111297000.0   
57    8343652013     Washington            10603000.0   -13271658.0   
87    9323722013     California            21560000.0   -15691000.0   
109  10919832013           Utah             2010000.0    -3602481.0   
117  11245242013     California              556000.0    -2063203.0   
142  14535932013        Montana            16459000.0    -3842372.0   
143  14639722013       New York              187000.0   -13037879.0   

     Debt to Equity  Total Liabilities  Total Revenue  Profit Margin  
18        -2.370953        592174000.0      719783000       0.320697  
57        -0.798921         16625000.0        8949401       0.448119  
87        -1.374036         30048000.0       37782000       0.505955  
109       -0.557949          6302000.0       17757388       0.732562  
117       -0.269484        

#### Create a new data frame for debt-to-income ratio

In [6]:
# -----------------------------
# Stepe 5. Create a new data frame that provides the debt-to-income ratio for every business in the data set. 
# Debt-to-income ratio is defined as long-term debt divided by revenue.
# Created dataframe df_dti for Debt to Income Ratio. 
# -----------------------------
# -----------------------------------------------------
# Function: create_debt_to_income
# Creates a DataFrame with Business ID and Debt-to-Income ratio
# Handles division-by-zero by setting result to NaN
# -----------------------------------------------------
def create_debt_to_income(df):
    df_dti = pd.DataFrame()
    df_dti["Business ID"] = df["Business ID"]

    # Use np.where to handle division by zero
    df_dti["Debt-to-Income"] = np.where(
        df["Total Revenue"] == 0,                # Uses 0 as the condition
        np.nan,                                  # If true then update to NaN (Not a Number)
        df["Total Long-term Debt"] / df["Total Revenue"]  # If not 0 then do the math. 
    )

    return df_dti


# Example usage
df_dti = create_debt_to_income(df)

print("\nDebt-to-Income Ratio DataFrame:")
print(df_dti.head())



Debt-to-Income Ratio DataFrame:
   Business ID  Debt-to-Income
0     41872013        0.123500
1     76232013        0.182665
2    160992013        0.049974
3    197452013        0.264664
4    241042013        0.036268


#### Concatenate the debt-to-income ratio to the original data frame. 

In [8]:
# -----------------------------
# Step 6. Concatenate the debt-to-income ratio data frame you created with the original data frame.
# -----------------------------
df_final = pd.concat([df, df_dti["Debt-to-Income"]], axis=1)

print("\nFinal DataFrame with Debt-to-Income Ratio:")
print(df_final.head())


Final DataFrame with Debt-to-Income Ratio:
   Business ID Business State  Total Long-term Debt  Total Equity  \
0     41872013       Kentucky            16889000.0    18046000.0   
1     76232013           Iowa             6252000.0    18293621.0   
2    160992013          Texas            19200000.0   177858000.0   
3    197452013       Delaware           117592000.0   278773000.0   
4    241042013       Illinois             4408000.0    52064000.0   

   Debt to Equity  Total Liabilities  Total Revenue  Profit Margin  \
0        0.935886         25986000.0      136753000       0.023663   
1        0.341758         14474000.0       34226553       0.265015   
2        0.107951         72787000.0      384196000       0.130413   
3        0.421820        558749000.0      444306000       0.196768   
4        0.084665         19898000.0      121541000       0.168305   

   Debt-to-Income  
0        0.123500  
1        0.182665  
2        0.049974  
3        0.264664  
4        0.036268  


### B.  Acknowledge sources, using in-text citations and references, for content that is quoted, paraphrased, or summarized.

No sources were used for this programming. 